In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
import time

In [3]:
tf.config.list_physical_devices('GPU')

[]

### About the mnist data set : http://yann.lecun.com/exdb/mnist/

In [4]:

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

template1 ='whole : {} | lev 1: {} | lev 2: {}'
# print(template1.format(len(x_train), len(x_train[0]), len(x_train[0][0]), len(x_train[0][0]), "\n", ("=")*30))

# print("This is whole:", "\n", ("-")*30, "\n", x_train, "\n", ("=")*30)
# print("This is lev 1:", " \n", ("-")*30, "\n", x_train[0], "\n", ("=")*30)
# print("This is lev 2:", " \n", ("-")*30, "\n", x_train[0][0], "\n", ("=")*30)
# print("This is lev 3:", " \n", ("-")*30, "\n", x_train[0][0][0], "\n", ("=")*30)

In [5]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

template1 ='whole: {} | lev 1: {} | lev 2: {} | lev 3: {}'
# print(template1.format(len(x_train), len(x_train[0]), len(x_train[0][0]), len(x_train[0][0][0]), "\n", ("-")*30))

# print("This is whole:", "\n", ("-")*30, "\n", x_train, "\n", ("=")*30)
# print("This is lev 1:", " \n", ("-")*30, "\n", x_train[0], "\n", ("=")*30)
# print("This is lev 2:", " \n", ("-")*30, "\n", x_train[0][0], "\n", ("=")*30)
# print("This is lev 3:", " \n", ("-")*30, "\n", x_train[0][0][0], "\n", ("=")*30)
# print("This is lev 4:", " \n", ("-")*30, "\n", x_train[0][0][0][0], "\n", ("=")*30)

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [7]:
# print(train_ds)
# print(test_ds)

In [8]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [10]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [11]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)

In [12]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [13]:
def train_model(epochs):
    startTime = time.time()
    for epoch in range(epochs):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

        for images, labels in train_ds:
#             train_step(images, labels)
#             to suppress the float warning, a better code is below , Henry-Jul. 2, 2020
            train_step(tf.cast(images, tf.float32), labels)

        for test_images, test_labels in test_ds:
#             test_step(test_images, test_labels)
            test_step(tf.cast(test_images, tf.float32), test_labels)

        template = 'Epoch {} :: Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        print(template.format(epoch + 1,
                                round(train_loss.result(), 3),
                                round(train_accuracy.result() * 100, 3),
                                round(test_loss.result(), 3),
                                round(test_accuracy.result() * 100), 3))
        
    total_run_time = time.time() - startTime
    return total_run_time

### Set epochs and iterations

In [14]:
epochs =1
iteration =3 
run_time = []

In [15]:
break-here # on purpose so the cpu kernel upon switching won't run & override the benchmark below

SyntaxError: invalid syntax (<ipython-input-15-f242f9b63921>, line 1)

### Kernel :: tf-cpu-env  :: using CPU

In [ ]:
# with cpu
for i in range(iteration):
    print(f'{("-")*30}\n Iteration # {i}')
    run_time.append(train_model(epochs))
    
for i in range(iteration):
    print(f'{("-")*30}\nGPU Run Summary ::\n Run # {i + 1} ::\
        {round(run_time[i]/60, 2)} {"minutes" if run_time[i]/60 > 1 else "minute"}')

In [ ]:
break-here # on purpose so the cpu kernel upon switching won't run & override the benchmark below

### Kernel :: tf-gpu-env  :: using GPU

In [ ]:
# with cpu
for i in range(iteration):
    print(f'{(" ")*30}\n {("=")*10} Iteration # {i} {("=")*10}')
    run_time.append(train_model(epochs))
    
for i in range(iteration):
    print(f'{("-")*30}\nGPU Run Summary ::\n Run # {i + 1} :: {run_time[i]/60} {"minutes" if run_time[i]/60 > 1 else "minute"}')